# Hands-on with Spark (Structured Streaming)

![spark](https://cdn-images-1.medium.com/max/300/1*c8CtvqKJDVUnMoPGujF5fA.png)

In the previous lesson, we learnt about Spark SQL, Dataframes and Pandas API. In this lesson, we will continue with the Structured Streaming.

Structured Streaming is a scalable and fault-tolerant stream processing engine built on the Spark SQL engine. You can express your streaming computation the same way you would express a batch computation on static data. The Spark SQL engine will take care of running it incrementally and continuously and updating the final result as streaming data continues to arrive.

You can use the Dataset/DataFrame API to express streaming aggregations, event-time windows, stream-to-batch joins, etc. The computation is executed on the same optimized Spark SQL engine. Finally, the system ensures end-to-end exactly-once fault-tolerance guarantees through checkpointing and Write-Ahead Logs. In short, Structured Streaming provides **fast, scalable, fault-tolerant, end-to-end exactly-once** stream processing without the user having to reason about streaming.

Internally, by default, Structured Streaming queries are processed using a micro-batch processing engine, which processes data streams as a series of small batch jobs thereby achieving end-to-end latencies as low as 100 milliseconds and exactly-once fault-tolerance guarantees.

## Installing and Initializing Spark

First, like previously, we'll need to install Spark and its dependencies:

1.   Java 8
2.   Apache Spark with Hadoop
3.   Findspark (used to locate the Spark in the system)


In [1]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

# set the options to connect to our Kafka cluster
options = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.scram.ScramLoginModule required username="YnJhdmUtZmlzaC0xMTQ2MyQSvwXBuLOQsV1W7YffuC8cDaZcA3fKQwakMhnQGgg" password="MDUxNjc4YzEtYzYxNy00NTE1LWEwNWYtMDBhODRlZmE0OGJm";',
    # "kafka.sasl.mechanism": "SCRAM-SHA-256",
    # "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'localhost:9092',
    "subscribe": 'pizza-orders',
}

In [ ]:
# # findspark is only required if you are using a standalone Spark installation (downloaded tar.gz)
# import findspark
# findspark.init()

The below cell may take a few minutes to run. It is slow because:
- Package downloading: Your PYSPARK_SUBMIT_ARGS includes a Kafka package that needs to be downloaded from Maven repositories on first run
- JVM cold start: Initial JVM startup is always slow
- Jupyter overhead: Running in a notebook adds some initialization overhead

In [2]:
# We will only use 2 cores below to speed up creation of the spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LearnSparkStreaming").master("local[2]").getOrCreate()

25/12/10 21:23:32 WARN Utils: Your hostname, DESKTOP-4VD7137 resolves to a loopback address: 127.0.1.1; using 172.26.8.82 instead (on interface eth0)
25/12/10 21:23:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/darwin/miniconda3/envs/kafka/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/darwin/.ivy2/cache
The jars for the packages stored in: /home/darwin/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8d44b8f6-9d04-490f-894c-783013d05797;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/sp

25/12/10 21:23:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

## Read and Analyze Kafka stream in "Batch" Mode

Let's start with reading and analyzing our `pizza-orders` kafka topic in the usual "batch" mode of Spark SQL and Dataframes. This is akin to the batch queries we did in the previous lesson. In this case we are taking the messages with the earliest to latest offsets of the topic as a single "batch".

In [4]:
pizza_df = spark.read.format('kafka')\
    .options(**options)\
    .load()

In [5]:
pizza_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
pizza_df.show()

+--------------------+--------------------+------------+---------+------+--------------------+-------------+
|                 key|               value|       topic|partition|offset|           timestamp|timestampType|
+--------------------+--------------------+------------+---------+------+--------------------+-------------+
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     0|2025-12-10 20:50:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     1|2025-12-10 20:50:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     2|2025-12-10 20:50:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     3|2025-12-10 20:50:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     4|2025-12-10 20:50:...|            0|
|[7B 22 73 68 6F 7...|[7B 22 69 64 22 3...|pizza-orders|        0|     5|2025-12-10 21:00:...|            0|
|[7B 22 73 68 6F 7.

In [7]:
pizza_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|{"shop": "Ill Mak...|{"id": 0, "shop":...|
|{"shop": "Ill Mak...|{"id": 1, "shop":...|
|{"shop": "Luigis ...|{"id": 2, "shop":...|
|{"shop": "Luigis ...|{"id": 3, "shop":...|
|{"shop": "Circula...|{"id": 4, "shop":...|
|{"shop": "Mammami...|{"id": 0, "shop":...|
|{"shop": "Its-a m...|{"id": 1, "shop":...|
|{"shop": "Mammami...|{"id": 2, "shop":...|
|{"shop": "Circula...|{"id": 3, "shop":...|
|{"shop": "Ill Mak...|{"id": 4, "shop":...|
|{"shop": "Its-a m...|{"id": 0, "shop":...|
|{"shop": "Luigis ...|{"id": 1, "shop":...|
|{"shop": "Luigis ...|{"id": 2, "shop":...|
|{"shop": "Mammami...|{"id": 3, "shop":...|
|{"shop": "Mammami...|{"id": 4, "shop":...|
+--------------------+--------------------+



In [8]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, StructType, ArrayType, StructField

In [9]:
pizza_schema = StructType([
  StructField("pizzaName", StringType()),
  StructField("additionalToppings", ArrayType(StringType())),
])

order_schema = StructType([
  StructField("address", StringType()),
  StructField("id", IntegerType()),
  StructField("name", StringType()),
  StructField("phoneNumber", StringType()),
  StructField("shop", StringType()),
  StructField("cost", DoubleType()),
  StructField("pizzas", ArrayType(pizza_schema)),
  StructField("timestamp", LongType()),
])

In [10]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [11]:
parsed_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phoneNumber: string (nullable = true)
 |    |-- shop: string (nullable = true)
 |    |-- cost: double (nullable = true)
 |    |-- pizzas: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- pizzaName: string (nullable = true)
 |    |    |    |-- additionalToppings: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- timestamp: long (nullable = true)



In [12]:
parsed_df.show(truncate=False)

+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|timestamp              |value                                                                                                                                                                                                                                                                                                                          |
+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

We can use _dot notation_ to select the field within a `Struct`:

In [13]:
parsed_df.select("value.cost").show()

+-----+
| cost|
+-----+
| 1.82|
| 2.71|
|34.83|
| 9.25|
| 5.84|
|21.33|
|11.46|
|43.16|
|40.62|
|22.93|
| 5.66|
|17.53|
|24.21|
|12.74|
| 1.67|
+-----+



Computing the "total revenue" per shop:

In [14]:
parsed_df.groupBy("value.shop").sum("value.cost").show(truncate=False)

+------------------------------------+-----------------------+
|shop                                |sum(value.cost AS cost)|
+------------------------------------+-----------------------+
|Ill Make You a Pizza You Cant Refuse|27.46                  |
|Luigis Pizza                        |85.82                  |
|Mammamia Pizza                      |78.89999999999999      |
|Its-a me! Mario Pizza!              |17.12                  |
|Circular Pi Pizzeria                |46.459999999999994     |
+------------------------------------+-----------------------+



> 1. Count the no. of orders by shop.
> 2. Compute the avg revenue by shop and sort by highest to lowest.

In [15]:
from pyspark.sql.functions import min, max

In [16]:
parsed_df.select(min("timestamp"), max("timestamp")).show(truncate=False)

+-----------------------+-----------------------+
|min(timestamp)         |max(timestamp)         |
+-----------------------+-----------------------+
|2025-12-10 20:50:03.411|2025-12-10 21:07:33.385|
+-----------------------+-----------------------+



## Read and Analyze Kafka stream in "Streaming" Mode

The key idea in Structured Streaming is to treat a live data stream as a table that is being continuously appended. This leads to a new stream processing model that is very similar to a batch processing model. You will express your streaming computation as standard batch-like query as on a static table, and Spark runs it as an incremental query on the *unbounded input table*. Let’s understand this model in more detail.

Consider the input data stream as the “Input Table”. Every data item that is arriving on the stream is like a new row being appended to the Input Table.

![concept](https://spark.apache.org/docs/latest/img/structured-streaming-stream-as-a-table.png)

A query on the input will generate the “Result Table”. Every trigger interval (say, every 1 second), new rows get appended to the Input Table, which eventually updates the Result Table. Whenever the result table gets updated, we would want to write the changed result rows to an external sink.

![result table](https://spark.apache.org/docs/latest/img/structured-streaming-model.png)

To illustrate the use of this model, let’s understand the model in context of a word count model. The first lines DataFrame is the input table, and the final wordCounts DataFrame is the result table. Note that the query on streaming lines DataFrame to generate wordCounts is exactly the same as it would be a static DataFrame. However, when this query is started, Spark will continuously check for new data from the socket connection. If there is new data, Spark will run an “incremental” query that combines the previous running counts with the new data to compute updated counts, as shown below.

![example](https://spark.apache.org/docs/latest/img/structured-streaming-example-model.png)




Event-time is the time embedded in the data itself. For many applications, you may want to operate on this event-time.

For example, if you want to get the number of events generated by IoT devices every minute, then you probably want to use the time when the data was generated (that is, event-time in the data), rather than the time Spark receives them.

This event-time is very naturally expressed in this model – each event from the devices is a row in the table, and event-time is a column value in the row. This allows window-based aggregations (e.g. number of events every minute) to be just a special type of grouping and aggregation on the event-time column – each time window is a group and each row can belong to multiple windows/groups. Therefore, such event-time-window-based aggregation queries can be defined consistently on both a static dataset (e.g. from collected device events logs) as well as on a data stream, making the life of the user much easier.

Furthermore, this model naturally handles data that has arrived later than expected based on its event-time. Since Spark is updating the Result Table, it has full control over updating old aggregates when there is late data, as well as cleaning up old aggregates to limit the size of intermediate state data.

In [17]:
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

In [18]:
pizza_df.isStreaming

True

In [19]:
pizza_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [20]:
parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value"))

In [21]:
parsed_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phoneNumber: string (nullable = true)
 |    |-- shop: string (nullable = true)
 |    |-- cost: double (nullable = true)
 |    |-- pizzas: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- pizzaName: string (nullable = true)
 |    |    |    |-- additionalToppings: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- timestamp: long (nullable = true)



In [ ]:
query = parsed_df \
    .writeStream \
    .format("console") \
    .start()

# query.awaitTermination() # stops the script from exiting. But this is not required in Jupyter because the kernel stays alive. In real Spark applications, you need awaitTermination() to keep the job running.
# query.isActive
# query.recentProgress
# query.stop()

25/12/10 21:37:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f40e8a2a-2e03-4306-b692-b45e6590538a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/12/10 21:37:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------------------+
|           timestamp|               value|
+--------------------+--------------------+
|2025-12-10 21:38:...|{409 Ward Stream\...|
+--------------------+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+--------------------+
|           timestamp|               value|
+--------------------+--------------------+
|2025-12-10 21:38:...|{272 Tammy Plains...|
+--------------------+--------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+--------------------+
|           timestamp|               value|
+--------------------+--------------------+
|2025-12-10 21:38:...|{3156 Dawson Knol...|
|2025-12-10 21:38:...|{0259 Brandon Val...|
|2025-12-10 21:38:...|{797 Shaw Junctio...|
+--------------------+--------------------+



In [23]:
query.stop()

In [24]:
query.recentProgress

[{'id': 'f88b3cd6-64e9-4989-8290-ce9fe0afcedc',
  'runId': '50dc4710-0cb7-47c6-b8ac-2ec7ee54a679',
  'name': None,
  'timestamp': '2025-12-10T13:37:41.526Z',
  'batchId': 0,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 0.0,
  'durationMs': {'addBatch': 2384,
   'getBatch': 58,
   'latestOffset': 3635,
   'queryPlanning': 455,
   'triggerExecution': 7343,
   'walCommit': 178},
  'stateOperators': [],
  'sources': [{'description': 'KafkaV2[Subscribe[pizza-orders]]',
    'startOffset': None,
    'endOffset': {'pizza-orders': {'0': 15}},
    'latestOffset': {'pizza-orders': {'0': 15}},
    'numInputRows': 0,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 0.0,
    'metrics': {'avgOffsetsBehindLatest': '0.0',
     'maxOffsetsBehindLatest': '0',
     'minOffsetsBehindLatest': '0'}}],
  'sink': {'description': 'org.apache.spark.sql.execution.streaming.ConsoleTable$@743aeabe',
   'numOutputRows': 0}},
 {'id': 'f88b3cd6-64e9-4989-8290-ce9fe0afcedc',

### In the next few cells, we will provide a complete example of counting the number of orders per pizza shop, as the orders stream in.

In [25]:
# First check if any streams are active
spark.streams.active

[]

In [26]:
# If there are any active streams, stop them
for q in spark.streams.active:
    print(f"Stopping query: {q.name}")
    q.stop()

In [27]:
# Complete example with counting
pizza_df = spark.readStream.format('kafka')\
    .options(**options)\
    .load()

parsed_df = pizza_df.select("timestamp", from_json(col("value").cast("string"), order_schema).alias("value")) #.groupBy("value.shop").count()

shop_counts = parsed_df.groupBy("value.shop").count()

# outputMode("complete") rewrites all aggregated results every trigger — good for full snapshots.
# outputMode("update") - only updated rows are written
query = shop_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

25/12/10 21:41:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-be26483e-1ef9-4d49-9343-bc7c8e4b57b7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/12/10 21:41:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|shop|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------+-----+
|          shop|count|
+--------------+-----+
|Mammamia Pizza|    1|
+--------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|        Luigis Pizza|    1|
|      Mammamia Pizza|    1|
|Its-a me! Mario P...|    1|
|        Marios Pizza|    2|
+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|        Luigis Pizza|    1|
|      Mammamia Pizza|    1|
|Its-a me! Mario P...|    1|
|        Marios Pizza|    3|
+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-----+
|                shop|count|
+--------------------+-----+
|        Luigis Pizza|    1|
|      Mammamia Pizza|    3|
|Its-a me! Mario P...|    3|
|        Marios Pizza|    3|
+--------------------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/darwin/miniconda3/envs/kafka/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/darwin/miniconda3/envs/kafka/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/darwin/miniconda3/envs/kafka/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [28]:
query.stop()

You have come to the end of this exercise.

To delete the Kafka topic `pizza-orders`, use the command below in your terminal:

`./kafka-topics.sh --delete --topic pizza-orders --bootstrap-server localhost:9092`

To exit Kafka in your terminal, type `exit`.